## 토픽 모델링
- 데이터에서 핵심 주제를 찾는 분석방법론
- 문서 내에 어떤 토픽이, 어떤 비율로 구성되어 있는지 확인

In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import re
import urllib.request

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
contents_pd = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DX_contents/kr_dx_concat_pre.csv', encoding='utf-8')
contents_pd.columns =['id', 'title', 'url', 'date', 'keyword', 'creator', 'view','likes', 'depart', 'depart1', 'language', 'text', 'new_text','key_words']

In [3]:
contents_pd

,id,title,url,date,keyword,creator,view,likes,depart,depart1,language,text,new_text,key_words
0,0,1강. 디지털 트랜스포메이션(DX)에 대한 이해,https://www.youtube.com/watch?v=TIW4r4FeOZE,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,2508,29.0,DX이해,DX기초,한국어,제 1강 ‘디지털 트랜스포메이션(DX)에 대한 이해‘ 강의입니다. \n1강에서는 디...,1강 디지털 트랜스포메이션DX에 대한 이해 제 1강 디지털 트랜스포메이션DX에 대한...,"디지털 트랜스포메이션, 생산성, 디지털 기술, 디지털 혁신, 산업DX, 디지털전환"
1,1,2강. 디지털 트랜스포메이션을 위한 사전준비,https://www.youtube.com/watch?v=zttI91TWqPs,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,764,11.0,DX이해,DX기초,한국어,제 2강 ‘디지털 트랜스포메이션을 위한 사전준비‘ 강의입니다. \n2강에서는 어떻게...,2강 디지털 트랜스포메이션을 위한 사전준비 제 2강 디지털 트랜스포메이션을 위한 사...,"디지털 트랜스포메이션, 디지털 혁신, 산업 DX, DX 전담조직PMO, KPC 디지..."
2,2,3강. 디지털 역량 진단,https://www.youtube.com/watch?v=dYr37tUnlvM,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,799,9.0,DX이해,DX기초,한국어,제 3강 ‘디지털 역량 진단‘ 강의입니다. \n3강에서는 디지털 트랜스포메이션을 위...,3강 디지털 역량 진단 제 3강 디지털 역량 진단 강의입니다 \n3강에서는 디지털 ...,"디지털 역량 진단, 디지털 트랜스포메이션, 디지털 비전, 리더십, 디지털 전략, 디..."
3,3,4강. 목표수립 및 방향설정,https://www.youtube.com/watch?v=5IekH0xnth8,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,504,9.0,DX이해,DX기초,한국어,제 4강 ‘목표수립 및 방향설정‘ 강의입니다. \n4강에서는 디지털 트랜스포메이션을...,4강 목표수립 및 방향설정 제 4강 목표수립 및 방향설정 강의입니다 \n4강에서는 ...,"디지털 트랜스포메이션, 디지털 비전, 디지털 리더십, 디지털 거버넌스, DX 교육,..."
4,4,5강. 디지털 비즈니스 모델 혁신,https://www.youtube.com/watch?v=7k4QEJNNoNs,2021.5.7,DX 표준 프레임워크 온라인교육,KPC디지털,405,5.0,DX이해,DX기초,한국어,제 5강 ‘디지털 비즈니스 모델 혁신‘ 강의입니다. \n5강에서는 디지털 비즈니스 ...,5강 디지털 비즈니스 모델 혁신 제 5강 디지털 비즈니스 모델 혁신 강의입니다 \n...,"디지털 비즈니스 모델, 디지털 비즈니스 모델 혁신, 디지털 트랜스포메이션, 비즈니스..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752,752,"잔디(JANDI), 잔디밭처럼 편안한 직장인 위한 소통 공간!”…토스랩 김대현 대표...",https://www.youtube.com/watch?v=WluXIGfs35g&pp...,2021.1.8,NaN,V-ON Venture must go ON,103,1.0,커뮤니케이션,그룹웨어,한국어,브이온(V-ON) 벤처·스타트업 대표 인터뷰\n\n“창업은 좀 더 나은 세상을 만드...,잔디JANDI 잔디밭처럼 편안한 직장인 위한 소통 공간토스랩 김대현 대표 인터뷰 브...,"잔디, 업무협업 툴, 실시간, 창업, 벤처스타트업, 미디어, 소통, 디지털 혁신"
753,753,14. 조직도 만들기(멤버 관리) ㅣ 잔디 튜토리얼,https://www.youtube.com/watch?v=unFIeTrHces&pp...,2020.12.22,NaN,JANDI,2347,1.0,커뮤니케이션,그룹웨어,한국어,멤버의 탈퇴/차단 등 멤버 관리가 궁금하신가요?\n00:00 PC에서 멤버 관리하기...,14 조직도 만들기멤버 관리 ㅣ 잔디 튜토리얼 멤버의 탈퇴차단 등 멤버 관리가 궁금...,"조직도, 멤버 관리, 무료 플랜, 유료 플랜, 모바일, 디지털 혁신"
754,754,협업툴 CTO의 협업 A to Z ⎮ 잔디가 자라는 방법 ep.2,https://www.youtube.com/watch?v=Ercl18BvRaU&pp...,2022.4.22,#JANDI,JANDI,608,11.0,커뮤니케이션,그룹웨어,한국어,#잔디 #JANDI #협업툴\n🌱 잔디가 자라는 방법\n✅ 언제 어디서나 가장 쉬운...,협업툴 CTO의 협업 A to Z 잔디가 자라는 방법 ep2 잔디 JANDI 협업...,"협업툴, 메신저, 업무용 커뮤니케이션 툴, 핵심 기능, 업무 몰입, 생산성, 협업,..."
755,755,"카카오톡? 잔디, 플로우, 네이버웍스, 노션... 우리 기업에 맞는 협업툴과 업무툴은?",https://www.youtube.com/watch?v=KTy1VZ9oMcw&pp...,2022.10.17,NaN,가인지TV,4066,66.0,커뮤니케이션,그룹웨어,한국어,✅ 경영벙커 강의파일과 자료 다운받기 👉 https://bit.ly/3CVqZX5\...,카카오톡 잔디 플로우 네이버웍스 노션 우리 기업에 맞는 협업툴과 업무툴은 경영벙커...,"카카오톡, 잔디, 플로우, 네이버웍스, 노션, 협업툴, 업무툴, 컨설팅회사, 경영 ..."


In [25]:
contents_pd['depart'].unique()

array(['DX이해', 'DX태도', '워크스킬', '데이터기술', '지능화기술', '자동화기술', '컴퓨팅기술',
       '최신기술활용역량', '커뮤니케이션'], dtype=object)

In [42]:
df1=contents_pd[contents_pd['depart']=='데이터기술']

In [43]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95 entries, 242 to 408
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         95 non-null     int64  
 1   title      95 non-null     object 
 2   url        95 non-null     object 
 3   date       95 non-null     object 
 4   keyword    75 non-null     object 
 5   creator    95 non-null     object 
 6   view       95 non-null     int64  
 7   likes      92 non-null     float64
 8   depart     95 non-null     object 
 9   depart1    95 non-null     object 
 10  language   95 non-null     object 
 11  text       95 non-null     object 
 12  new_text   95 non-null     object 
 13  key_words  95 non-null     object 
dtypes: float64(1), int64(2), object(11)
memory usage: 11.1+ KB


In [44]:
df1['key_words'][:5]

242    Orange3, 머신러닝, 데이터 마이닝, 빅데이터, 데이터 분석, 코딩, Pyth...
243    Orange, 데이터 분석, 시계열 분석, ARIMA, 위젯, 마우스 클릭, 마우스...
244    Orange, 데이터 마이닝, 클러스터링, 분류법, 이미지 데이터, 오픈 소스, 데...
245                  Orange3, 데이터 분석, 머신 러닝, 인공지능, 집값 예측
246                   STEP Orange, 정형 데이터 분석, 데이터 마이닝 도구
Name: key_words, dtype: object

### Sklearn으로 LDA

In [45]:
# 상위 1,000개의 단어를 보존
vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000)
X = vectorizer.fit_transform(df1['key_words'])

# TF-IDF 행렬의 크기 확인
print('TF-IDF 행렬의 크기 :', X.shape)

TF-IDF 행렬의 크기 : (95, 567)


In [46]:
lda_model = LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

In [47]:
lda_top = lda_model.fit_transform(X)

In [48]:
print(lda_model.components_)
print(lda_model.components_.shape)

[[0.79187797 0.7643112  0.95055429 ... 0.82374461 0.81303588 0.75108979]
 [0.66643395 0.77041779 0.87714267 ... 0.7486447  0.87031294 0.84350349]
 [0.72953679 0.92016692 0.79993756 ... 0.83797509 0.86772354 0.8358858 ]
 ...
 [0.8364506  0.87869292 0.77359746 ... 0.72486651 0.84087724 0.84203872]
 [0.77817889 0.88750857 0.80700707 ... 0.67445668 0.86053935 0.92305303]
 [0.86193532 1.01597139 0.70628016 ... 0.8188294  0.80558923 0.79410204]]
(10, 567)


In [51]:
# 단어 집합. 1,000개의 단어가 저장됨.
terms = vectorizer.get_feature_names_out()

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda_model.components_,terms)

Topic 1: [('자동채우기', 1.14), ('세로막대', 1.13), ('클라우드', 1.07), ('혁신적인', 1.07), ('다층', 1.06)]
Topic 2: [('엑셀', 1.44), ('관계형성', 1.23), ('혁신', 1.23), ('디노마케팅', 1.18), ('db', 1.18)]
Topic 3: [('그래프', 1.24), ('power', 1.13), ('구성', 1.1), ('산업', 1.1), ('마우스', 1.08)]
Topic 4: [('analysis', 1.1), ('집계', 1.08), ('t아카데미', 1.05), ('방법', 1.05), ('dimension', 1.04)]
Topic 5: [('프론트엔드', 1.14), ('crm', 1.09), ('인프런', 1.06), ('그룹화', 1.06), ('web', 1.05)]
Topic 6: [('데이터', 1.46), ('엑셀', 1.26), ('분석', 1.19), ('erd', 1.16), ('시각화', 1.14)]
Topic 7: [('sns', 1.15), ('데이터베이스', 1.09), ('elastic', 1.07), ('erd', 1.06), ('프로세스', 1.06)]
Topic 8: [('빅쿼리', 1.13), ('knime', 1.11), ('중고차', 1.1), ('머신러닝', 1.08), ('데이터', 1.06)]
Topic 9: [('수집', 1.1), ('시간', 1.09), ('원칙', 1.09), ('web', 1.08), ('매장', 1.08)]
Topic 10: [('데이터', 1.62), ('처리', 1.15), ('orange', 1.15), ('프로그램', 1.15), ('딥러닝', 1.13)]
